In [ ]:
from PIL import Image, ImageEnhance, ImageFilter
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow import keras

In [ ]:
path = 'image.png' # insert path to image here

In [ ]:
def image_to_points(image_path):

    img = Image.open(image_path).convert('L')
    img_array = np.array(img)

    points = []
    for y, row in enumerate(img_array):
        for x, color in enumerate(row):
            if color <= 100:
                points.append((x, y))

    return points

points = np.array(image_to_points(path))

In [ ]:
from sklearn.cluster import DBSCAN

sentence = DBSCAN(eps=40, min_samples=70).fit(points)
sentence.labels_

In [ ]:
print(np.unique(sentence.labels_))

In [ ]:
sentence_labels = np.unique(sentence.labels_)

In [ ]:
def sort_centers(clusters, labels, points):
  centers = []
  empty = 0
  for i in range(len(labels)):
    letter = points[clusters.labels_.ravel() == i]
    if(len(letter) <= 0):
      empty += 1
      continue
    centers.append(np.mean(letter[:, 0]))

  num_centers = labels.shape[0] - empty
  centers = np.array(centers).reshape(num_centers, 1)
  index = np.arange(num_centers).reshape(num_centers, 1)
  x_vals = np.append(centers, index, 1)
  sorted_centers = x_vals[x_vals[:, 0].argsort()]

  return sorted_centers

In [ ]:
def preprocess_img(image, target_size=(28, 28)):
    img = cv2.bitwise_not(image)
    kernel = np.ones((7, 7), np.uint8)  # Kernel size determines the thickness increase
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
    img = img / 255.0
    img = img.reshape(1, target_size[0], target_size[1], 1).astype('float32')
    return img

In [ ]:
def display_word(images, num_imgs=7):
    fig, axes = plt.subplots(1, num_imgs, figsize=(7, 2))

    for i in range(num_imgs):
      img = preprocess_img(images[i])
      img_reshaped = img.reshape(28, 28)
      if(num_imgs == 1):
        axes.imshow(img_reshaped, cmap='gray')
        axes.axis('off')
      else:
        axes[i].imshow(img_reshaped, cmap='gray')
        axes[i].axis('off')

    plt.show()

In [ ]:
def predict_word(word, model, word_points):
  labels = np.unique(word.labels_)

  sorted_centers = sort_centers(word, labels, word_points)
  prediction = ""

  images = []

  for i in sorted_centers:
    letter = word_points[word.labels_.ravel() == (i[1])]
    min_x = int(min(letter[:, 0]))
    range_x = int(max(letter[:, 0]) - min_x)
    min_y = int(min(letter[:, 1]))
    range_y = int(max(letter[:, 1]) - min_y)
    dim = max(range_x, range_y)
    padding = dim / 3
    dim = int(max(range_x, range_y) + 2 * padding)
    padding_x = int((dim - range_x) / 2)
    padding_y = int((dim - range_y) / 2)
    im = Image.new(mode = "RGB", size = (int(range_x) + 2 * padding_x, int(range_y) + 2 * padding_y), color = (255, 255, 255))

    one_letter = np.array(im)

    for point in letter:
      one_letter[int(point[1] - min_y)][int(point[0]- min_x)] = [0, 0, 0]

    blurred_letter = cv2.GaussianBlur(one_letter, (3, 3), 0)
    img = Image.fromarray(blurred_letter, 'RGB')

    square_im = Image.new(mode = "RGB", size = (dim, dim), color = (255, 255, 255))
    square_im.paste(img, (padding_x, padding_y))
    square_im = square_im.resize((128,128), resample = Image.LANCZOS)

    arr = np.asarray(square_im)[:, :, 1]
    images.append(arr)
    preproc = preprocess_img(arr)

    pred = model.predict(preproc)
    letter_pred = chr(np.argmax(pred, axis=1)[0] + 65)
    prediction += letter_pred

  display_word(images, len(sorted_centers))
  return prediction


In [ ]:
model = keras.models.load_model("hw_final.keras")

In [ ]:
words = sort_centers(sentence, sentence_labels, points)
sentence_pred = ""
for word in words[:,1]:
  word_points = points[sentence.labels_.ravel() == (word)]
  word_cluster = DBSCAN(eps=3, min_samples=10).fit(word_points)
  pred = predict_word(word_cluster, model, word_points)
  sentence_pred += pred + " "
  print(pred + " ")

In [ ]:
print(sentence_pred)